# Ejemplo Python-SQL

Considere las siguientes relaciones:
- Flights(flno: integer, origin: string, destination: string, distance: integer, departs: time, arrives: time, price: integer)
- Aircraft(aid: integer, aname: string, cruisingrange: integer)
- Employees(eid: integer, ename: string, salary: integer)
- Certified(eid, aid)

Considere que la relación _Employees_ describe tanto a pilotos como a otro tipo de trabajador, que cada piloto está certificado para volar algún avión, y que solo los pilotos están certificados para volar.

En base a esto, construya una base de datos en SQLite con este esquema, pueble las tablas con los datos contenidos en los archivos .txt correspondientes, y escriba las consultas SQL para responder a las preguntas que se indican a continuación:
- Encuentre los nombres de los pilotos certificados para volar algún avión Boeing.
- Encuentre los nombres de los pilotos que están certificados para volar aviones con autonomía de más de 1000 millas.
- Por cada piloto que está certificado para volar más de tres aviones, encuentre el _id_ y la autonomía del avión con máxima autonomía para el que está certificado.
- Encuentre los _id_ de los aviones que pueden ser usados en rutas de Los Ángeles a Chicago.
- Encuentre los nombres de los pilotos cuyo sueldo es menor que el precio del vuelo más barato de Los Ángeles a Honolulu.
- Encuentre los nombres de los aviones cuyos pilotos certificados tienen todos sueldos mayores a US$ 80.000.
- Para todos los aviones con autonomía superior a 1000 millas, encuentre el nombre del avión y el sueldo promedio de los pilotos certificados para volarlo.

## Creación de tablas y carga de datos

In [ ]:
import sqlite3
import csv

In [ ]:
connection = sqlite3.connect('example.db') 
cursor = connection.cursor()

In [ ]:
cursor.execute("CREATE TABLE Flights(flno INTEGER PRIMARY KEY, origin TEXT, destination TEXT, \
                distance INTEGER, departs DATE, arrives DATE, price REAL)")
with open('flights.txt') as f:
    for line in csv.reader(f):
        cursor.execute('INSERT INTO flights VALUES (?,?,?,?,?,?,?)', tuple(line))

In [ ]:
cursor.execute("CREATE TABLE Aircraft(aid INTEGER PRIMARY KEY, aname TEXT, cruisingrange INTEGER)")
with open('aircraft.txt') as f:
    for line in csv.reader(f):
        cursor.execute('INSERT INTO aircraft VALUES (?,?,?)', tuple(line))

In [ ]:
cursor.execute("CREATE TABLE Employees(eid INTEGER PRIMARY KEY, ename TEXT, salary REAL)")
with open('employees.txt') as f:
    for line in csv.reader(f):
        cursor.execute('INSERT INTO employees VALUES (?,?,?)', tuple(line))

In [ ]:
cursor.execute("CREATE TABLE Certified(eid INTEGER, aid INTEGER, PRIMARY KEY(eid,aid), \
                FOREIGN KEY(eid) REFERENCES employees, FOREIGN KEY(aid) REFERENCES aircraft)")
with open('certified.txt') as f:
    for line in csv.reader(f):
        cursor.execute('INSERT INTO certified VALUES (?,?)', tuple(line))

In [ ]:
connection.commit()

In [ ]:
for e in cursor.execute('SELECT ename, salary \
                         FROM employees \
                         ORDER BY ename'):
    print(e)

In [ ]:
connection.close()

## Consultas

In [ ]:
# Encuentre los nombres de los pilotos certificados para volar algún avión Boeing.

connection = sqlite3.connect('example.db') 
cursor = connection.cursor()
cursor.execute("SELECT DISTINCT E.ename \
                FROM Employees E, Certified C, Aircraft A \
                WHERE E.eid = C.eid AND \
                C.aid = A.aid AND \
                A.aname LIKE '%Boeing%'")
for r in cursor:
    print(r[0])
connection.close()

In [ ]:
# Encuentre los nombres de los pilotos que están certificados para volar aviones con autonomía de más de 1000 millas.

connection = sqlite3.connect('example.db') 
cursor = connection.cursor()
cursor.execute('SELECT DISTINCT E.ename \
                FROM Employees E, Certified C, Aircraft A \
                WHERE C.aid = A.aid AND E.eid = C.eid AND A.cruisingrange > 1000')
for r in cursor:
    print(r[0])
connection.close()

In [ ]:
# Encuentre los nombres de los pilotos que están certificados para volar SOLO aviones con autonomía de más de 1000 millas.

connection = sqlite3.connect('example.db') 
cursor = connection.cursor()
cursor.execute('SELECT E.ename \
                FROM Employees E, Certified C, Aircraft A \
                WHERE C.aid = A.aid AND E.eid = C.eid \
                GROUP BY E.eid, E.ename \
                HAVING A.cruisingrange > 1000')
for r in cursor:
    print(r[0])
connection.close()

In [ ]:
# Por cada piloto que está certificado para volar más de tres aviones, encuentre el _id_ del piloto y la autonomía del avión con máxima autonomía para el que está certificado.

connection = sqlite3.connect('example.db') 
cursor = connection.cursor()
cursor.execute('SELECT C.eid, MAX (A.cruisingrange) \
                FROM Certified C, Aircraft A \
                WHERE C.aid = A.aid \
                GROUP BY C.eid \
                HAVING COUNT (*) > 3')
for r in cursor:
    print(r[0], r[1])
connection.close()

In [ ]:
# Encuentre los _id_ y nombres de los aviones que pueden ser usados en rutas de Los Ángeles a Chicago.

connection = sqlite3.connect('example.db') 
cursor = connection.cursor()
cursor.execute("SELECT A.aid, A.aname \
                FROM Aircraft A \
                WHERE A.cruisingrange > (SELECT MIN (F.distance) \
                                         FROM Flights F \
                                         WHERE F.origin = 'Los Angeles' AND F.destination = 'Chicago')")
for r in cursor:
    print(r[0], r[1])
connection.close()

In [ ]:
# Encuentre los nombres de los pilotos cuyo sueldo es menor que el precio del vuelo más barato de Los Ángeles a Honolulu.

connection = sqlite3.connect('example.db') 
cursor = connection.cursor()
cursor.execute("SELECT DISTINCT E.ename \
                FROM Employees E \
                WHERE E.salary < (SELECT MIN (F.price) \
                                  FROM Flights F \
                                  WHERE F.origin = 'Los Angeles' AND F.destination = 'Honolulu')")
for r in cursor:
    print(r[0])
connection.close()


In [ ]:
# Encuentre los nombres de los aviones cuyos pilotos certificados tienen todos sueldos mayores a US$ 80.000.

connection = sqlite3.connect('example.db') 
cursor = connection.cursor()
cursor.execute("SELECT DISTINCT A.aname \
                FROM Aircraft A \
                WHERE A.Aid IN (SELECT C.aid \
                                FROM Certified C, Employees E \
                                WHERE C.eid = E.eid AND NOT EXISTS (SELECT * \
                                                                    FROM Employees E1 \
                                                                    WHERE E1.eid = E.eid AND E1.salary < 80000))")
for r in cursor:
    print(r[0])
connection.close()

In [ ]:
# Para todos los aviones con autonomía superior a 1000 millas, encuentre el nombre del avión y el sueldo promedio de los pilotos certificados para volarlo.

connection = sqlite3.connect('example.db') 
cursor = connection.cursor()
cursor.execute('SELECT Temp.name, Temp.AvgSalary \
                FROM (SELECT A.aid, A.aname AS name, AVG (E.salary) AS AvgSalary \
                      FROM Aircraft A, Certified C, Employees E \
                      WHERE A.aid = C.aid AND C.eid = E.eid AND A.cruisingrange > 1000 \
                      GROUP BY A.aid, A.aname) AS Temp')
for r in cursor:
    print(r[0],r[1])
connection.close()